# 1、ConversationTokenBufferMemory的使用
举例1

In [ ]:
from langchain_classic.memory import ConversationTokenBufferMemory, ConversationSummaryMemory
from langchain_community.chat_models.tongyi import ChatTongyi
from dotenv import load_dotenv

load_dotenv()

# 2.创建大模型
llm = ChatTongyi()
# 3.定义ConversationTokenBufferMemory对象
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=10  # 设置token上限
)
# 添加对话
memory.save_context({"input": "你好吗？"}, {"output": "我很好，谢谢！"})
memory.save_context({"input": "今天天气如何？"}, {"output": "晴天，25度"})
# 查看当前记忆
print(memory.load_context({}))

# 2、ConversationSummaryMemory的使用

In [ ]:
from langchain_classic.memory.summary import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm=llm,
)
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手小智"})
memory.save_context({"input": "初次对话，你能介绍一下你自己吗？"}, {"output": "当然可以了。我是一个无所不能的小智。"})
# 5.读取消息（总结后的）
print(memory.load_memory_variables({}))

如果实例化ConversationSummaryMemory前，已经有历史消息，可以调用from_messages()实例化

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_classic.memory.summary import ConversationSummaryMemory
from langchain_community.chat_models.tongyi import ChatTongyi
from dotenv import load_dotenv

load_dotenv()

llm = ChatTongyi()
# 3.假设原始消息
history = InMemoryChatMessageHistory()
history.add_user_message("你好，你是谁？")
history.add_ai_message("我是AI助手小智")
# 4.创建ConversationSummaryMemory

memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    chat_memory=history,
)
print(memory.load_memory_variables({}))

In [ ]:
memory.save_context(inputs={"human": "我的名字叫小明"}, outputs={"ai": "很高兴认识你"})

print(memory.load_memory_variables({}))

ConversationSummaryBufferMemory的使用

In [ ]:
from langchain_classic.chains.llm import LLMChain
from langchain_classic.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain_classic.memory import ConversationEntityMemory
from langchain_community.chat_models.tongyi import ChatTongyi

# 初始化大语言模型
llm = ChatTongyi()
# 使用LangChain为实体记忆设计的预定义模板
prompt = ENTITY_MEMORY_CONVERSATION_TEMPLATE
# 初始化实体记忆
memory = ConversationEntityMemory(llm=llm)
# 提供对话链
chain = LLMChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm),
    verbose=True,  # 设置为True可以看到链的详细推理过程
)
# 进行几轮对话，记忆组件会在后台自动提取和存储实体信息
chain.invoke(input="你好，我叫蜘蛛侠。我的好朋友包括钢铁侠、美国队长和绿巨人。")
chain.invoke(input="我住在纽约。")
chain.invoke(input="我使用的装备是由斯塔克工业提供的。")
# 查询记忆体中存储的实体信息
print("\n当前存储的实体信息:")
print(chain.memory.entity_store.store)
# 基于记忆进行提问

In [ ]:
answer = chain.invoke(input="你能告诉我蜘蛛侠住在哪里以及他的好朋友有哪些吗？")
print("\nAI的回答:")
print(answer)


ConversationKGMemory的使用

In [ ]:
from langchain_community.memory.kg import ConversationKGMemory
from langchain_community.chat_models.tongyi import ChatTongyi
from dotenv import load_dotenv

load_dotenv()

llm = ChatTongyi()
# 3.定义ConversationKGMemory对象
memory = ConversationKGMemory(llm=llm)
# 4.保存会话
memory.save_context({"input": "向山姆问好"}, {"output": "山姆是谁"})
memory.save_context({"input": "山姆是我的朋友"}, {"output": "好的"})
# 5.查询会话
memory.load_memory_variables({"input": "山姆是谁"})

In [ ]:
memory.get_knowledge_triplets("山姆最喜欢的颜色是红色")
# print(memory)